In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/archive")

In [0]:
df_geolocation = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/archive/olist_geolocation_dataset.csv")

df_geolocation.show()

In [0]:
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_unique_id", StringType(), True),
    StructField("customer_zip_code_prefix", StringType(), True),
    StructField("customer_city", StringType(), True),
    StructField("customer_state", StringType(), True),
])

In [0]:
df_geolocation  = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \ 
    .schema(schema) \
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/archive/olist_geolocation_dataset.csv") \
    .writeStream \
    .format("delta") \
    .option("schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/geolocation") \
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/geolocation") \
    .table("bronze.geolocation")